In [153]:
from pptx import Presentation
import json
from datetime import datetime


In [154]:
ppt_template_path = "C:/Users/hamada.saleh/Desktop/myPPT_template.pptx"

In [155]:
# Helpers
def read_pptx(filename):
    """
    Read a PowerPoint presentation (.pptx) file and return the presentation object.

    Parameters:
    - filename (str): The name of the PowerPoint (.pptx) file to read.

    Returns:
    - prs (Presentation): The presentation object.
    """
    # Create a presentation object
    prs = Presentation(filename)

    return prs
def add_title(title: str, slide, title_id: int) -> None:
    """
    Add a title to a given slide.

    Parameters:
    - slide (pptx.slides.Slide): The slide to which the title will be added.
    - title (str): The text of the title.

    Returns:
    - None: This function does not return any value. It modifies the slide object.
    """
    title_shape = slide.shapes[title_id]
    title_shape.text = title
    
def set_title_slide(prs: "Presentation", title: str, subtitle: str, date: str) -> "Presentation":
    """
    This function sets the title slide of a PowerPoint presentation.

    Parameters:
    prs (Presentation): The presentation object to which the title slide will be added.
    title (str): The main title of the presentation.
    subtitle (str): The subtitle of the presentation.
    date (str): The date to be displayed on the title slide.

    Returns:
    prs (Presentation): The updated presentation object with the added title slide.

    Note:
    - The function assumes that the presentation template has a title slide layout at index 0.
    - The function accesses the shapes of the title slide by their index. This may not work if the
      placeholder names from the layout are reset when the actual slide is instantiated.
    """
    # Title slide layout
    title_slide_layout = prs.slide_layouts[0]

    # Add a title slide based on the title slide layout
    title_slide = prs.slides.add_slide(title_slide_layout)

    # TODO: Placeholder names from layout seem to be reset
    # TODO: when actual slide is instantiated
    # Accessing shapes by index (assuming placeholders are not renamed)
    title_shape0 = title_slide.shapes[0]
    title_shape1 = title_slide.shapes[1]
    title_shape2 = title_slide.shapes[2]

    # Set the text for each shape
    title_shape0.text = date
    title_shape1.text = title
    title_shape2.text = subtitle

    return prs

[Modeling the spatial dependence of floods](https://hess.copernicus.org/articles/23/107/2019/hess-23-107-2019.pdf)

In [156]:
# Presentation title
TITLE = "Modelling flood-related physical risks"
# Presentation subtitle
SUBTITLE = "The importance of integrating spatial dependence for reliable damage estimates"

# Get the current date
current_date = datetime.now()

# Format the date as DD/MM/YYYY
formatted_date = current_date.strftime("%d/%m/%Y")

# Read powerpoint template file

In [157]:
prs = read_pptx(ppt_template_path)

In [158]:
# Available slides in presentation template
[slide_layout.name for slide_layout in prs.slide_layouts]

['Titre',
 'Section',
 'Sommaire',
 'Contenu classique',
 'Contenu classique avec note',
 '2 contenus',
 '2 contenus + couleur droite',
 '1_2 contenus + 2 couleurs',
 '2 contenus + 2 couleurs',
 'Graphe et commentaire',
 '2 Graphes et commentaires',
 '2 Graphes et 1 commentaire',
 'Titre seul',
 'Annexe']

## Title slide

In [159]:
set_title_slide(prs=prs, title=TITLE, subtitle=SUBTITLE, date=formatted_date)

In [160]:
prs.save("pladifes.pptx")

# Outline slide

## Content slide

In [ ]:
# Get "Contenu classique" slide layout index
cc_id = lay2idx["Contenu classique"]
# Create new slide
slide = layouts[cc_id]
# Mapping between slide shapes and indices
shape2idx = {s.name: i for i, s in enumerate(slide.shapes)}

In [ ]:
# index of title element
title_id = shape2idx["Titre 1"]

# Define title
title = "Hello, World!"
# change title text
add_title(title=title, slide=slide, title_id=title_id)
prs.save("pladifes.pptx")

In [ ]:
print(dir(slide))

In [ ]:
for s in slide.shapes:
    print(s.name)

In [ ]:
shape2idx

In [ ]:
prs2 = Presentation()

In [ ]:
slide = prs2.slide_layouts[1]
prs2.slides.add_slide(slide)
prs2.save("pladifes2.pptx")

In [ ]:
for s in slide.shapes:
    print(s.name)

In [ ]:
with open("slides.json", 'r') as file:
    data = json.load(file)

In [ ]:
data

In [ ]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN

def add_image_and_text_slide(prs, image_path, title, content):
    # Choose a slide layout with content and picture placeholder
    slide_layout = prs.slide_layouts[5]  # 5 corresponds to 'Title Slide with Picture Placeholder'

    # Add the new slide to the presentation
    slide = prs.slides.add_slide(slide_layout)

    # Set title
    title_shape = slide.shapes.title
    title_shape.text = title

    # Add image on the left side
    left_inch = Inches(0.5)
    top_inch = Inches(1.5)
    pic = slide.shapes.add_picture(image_path, left_inch, top_inch, width=Inches(4))

    # Add text on the right side
    text_box = slide.shapes.add_textbox(Inches(5), top_inch, Inches(3), Inches(5))
    text_frame = text_box.text_frame
    p = text_frame.add_paragraph()
    p.text = content
    p.alignment = PP_ALIGN.LEFT
    p.font.size = Pt(18)


In [ ]:
# Add a slide with image on the left and text on the right
add_image_and_text_slide(prs, 'earth.jpg', 'Slide Title', 'Text content goes here.')
# Save the presentation
prs.save("slidego_pres.pptx")